# Create Custom Convolution Function

In [1]:
import numpy as np

In [2]:
from imp import reload

In [3]:
# Instead of weightedIterativeStatistic, let's just use the 
# calc_wfm function we created to get a good estimate.

In [4]:
# Let's try and implement this in numpy/scipy instead.

In [5]:
import for_upload.wFmConv as wfc

In [6]:
import customConv as ccv

## Read in Input

In [7]:
from process_radioML_data import *

In [8]:
X, lbl, snrs, classes = read_in_RML()

In [9]:
X_train, Y_train, X_test, Y_test = partition_train_test(X, \
                    lbl, classes, maxtrain=1000, maxtest=500)

In [10]:
X_train.shape, Y_train.shape, X_test.shape, Y_test.shape

((1000, 2, 128), (1000, 11), (500, 2, 128), (500, 11))

## Put something through a convolutional layer

In [11]:
point_list = X_train[10]

In [12]:
outdim = 126

In [43]:
num_channels = 2

In [44]:
window_length = 3

In [65]:
filter_array = ccv.make_filters(outdim, num_channels, window_length)

In [66]:
filter_array.shape

(126, 2, 3)

In [86]:
reload(ccv)

<module 'customConv' from '/home/allisonrmcalister/customConv.py'>

In [87]:
convolved = ccv.complex_conv(point_list, outdim, filter_array)

In [88]:
convolved

array([[ 6.51614780e-03,  2.51579913e-03,  1.71970832e-03,
         2.28791990e-03, -1.52091496e-03, -3.44860181e-03,
        -6.98360695e-03, -6.47597201e-03, -7.04007549e-03,
        -6.70658983e-03, -5.52810729e-03, -3.63644329e-03,
        -1.56215337e-03,  1.70624581e-03,  3.90356523e-03,
         5.97775262e-03,  7.31090549e-03,  7.88181648e-03,
         7.62783224e-03,  6.71994220e-03,  5.36846835e-03,
         3.89049016e-03,  4.78643241e-03,  1.63341942e-03,
         1.31007470e-03,  1.67436490e-03,  2.29637204e-03,
         4.68709340e-03,  5.83816506e-03,  7.51472265e-03,
         8.84112716e-03,  1.14084652e-02,  9.70487855e-03,
         9.01245512e-03,  7.57436967e-03,  5.57227712e-03,
         3.21448897e-03,  2.69793569e-04, -1.52638159e-03,
        -3.40897450e-03, -2.43393969e-03, -4.25341214e-03,
        -5.42208645e-03, -4.71771547e-03, -3.71283540e-03,
        -4.33713905e-03, -2.28347606e-03, -1.74977037e-03,
        -1.72306085e-03, -2.17734231e-03, -3.13348859e-0

In [95]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

In [96]:
def relu(x):
    x = (x > 0) * x
    return x

In [97]:
def grad_loss(prediction, labels):
    label_reshape = ccv.to_categorical(labels)
    grad_loss_out = []
    for i in range(len(prediction)):
        grad_loss_out.append(2*(prediction[i] - label_reshape[i]))
    return np.array(grad_loss_out)

In [94]:
def single_layer_backward_propagation(dA_curr, W_curr, b_curr, Z_curr, \
                                     A_prev, activation='relu'):
    m = A_prev.shape[1]
    
    if activation is 'relu':
        backward_activation_func = relu_backward
    elif activation is 'sigmoid':
        backward_activation_func = sigmoid_backward
    else:
        raise Exception('Non-supported activation function')
    
    dZ_curr = backward_activation_func(dA_curr, Z_curr)
    dW_curr = np.dot(dZ_curr, A_prev.T) / m
    db_curr = np.sum(dZ_curr, axis=1, keepdims=True) / m
    dA_prev = np.dot(W_curr.T, dZ_curr)
    
    return dA_prev, dW_curr, db_curr

In [93]:
def convolutional_input_backpropagation(actual_layer_outputs, \
    previous_layer_kernel_weights, previous_layer_bias,\
    previous_layer_outputs, grad_loss_outputs):
    # Calculating the weight gradients
    
    # Step 1 - Calculate the gradient of the loss w.r.t. the layers node outputs
    # Step 2 - Calculate the gradient of each node w.r.t. each z,
    # where output[i] = sigmoid(z[i])
    
    grad_outputs_z = actual_layer_outputs * (1 - actual_layer_outputs)
    
    # Step 3 - Calculate the gradient of the loss w.r.t. each
    # kernel weight using the chain rule
    batches = len(actual_layer_outputs)
    n_kernels = previous_layer_kernel_weights.shape[-1]
    k_row = previous_layer_kernel_weights.shape[0]
    k_col = previous_layer_kernel_weights.shape[1]
    
    outputs_row = actual_layer_outputs.shape[1]
    outputs_col = actual_layer_outputs.shape[2]
    
    grad_k = np.empty((batches, k_row, k_col, n_kernels))
    for batch in range(batches):
        for nk in range(n_kernels):
            for i in range(k_row):
                for j in range(k_col):
                    grad_k[batch,i,j] = np.sum(\
                        (grad_loss_outputs[batch][:,:,nk] *\
                        grad_outputs_z[batch][:,:,nk])\
                    * previous_layer_outputs[batch][i:i+outputs_row:j+outputs_col])
                                               
    # Step 4 - Calculate the average of the gradients
    grad_loss_kernel = np.empty((k_row, k_col, n_kernels))
    for nk in range(n_kernels):
        for i in range(k_row):
            for j in range(k_col):
                grad_loss_kernel[i,j,nk] = np.mean(grad_k[:,i,j,nk])
    return grad_loss_kernel